Curves can be expressed parametrically by representing each coordinate with an explicit equation in terms of a new parameter. The following code defines a parametric expression for a 2D curve, based on a parameter $t$ and two extra $\alpha$ and $\beta$ symbolic values. Although for this expression the derivatives can be found analytically, we use the sympy symbolic differentiation for the examples.

In [ ]:
from sympy import *    

In [2]:
# symbol definition

t = symbols('t')
a = symbols('alpha')
b = symbols('beta')

# parametric function

fx = cos(t*a) + 1/3
fy = sin(t+b)

display(fx)
display(fy)

cos(alpha*t) + 0.333333333333333

sin(beta + t)

In [3]:
import numpy as np

In [4]:
# evaluation & derivatives

In [5]:
def f0_parametric(t1, alpha1, beta1):

    return np.array( [fx.subs({t:t1, a:alpha1}).evalf() , 
                      fy.subs({t:t1, b:beta1} ).evalf() ],  
                      dtype=np.float64 )

In [6]:
def f1_parametric(t1, alpha1, beta1):

    return np.array( [fx.diff(t, 1).subs({t:t1, a:alpha1}).evalf() , 
                      fy.diff(t, 1).subs({t:t1, b:beta1} ).evalf() ] , 
                      dtype=np.float64)

In [7]:
def f2_parametric(t1, alpha1, beta1):

    return np.array( [fx.diff(t, 2).subs({t:t1, a:alpha1}).evalf() , 
                      fy.diff(t, 2).subs({t:t1, b:beta1} ).evalf() ] , 
                      dtype=np.float64)

In [8]:
# define objective function: 

In [9]:
import math

def curvature(t, a1, b1):
      
        v = f1_parametric(t, a1, b1)

        a = f2_parametric(t, a1, b1)

        w = np.cross(v,a)

        magnitude = np.linalg.norm(w)

        speed = np.linalg.norm(v)

        return magnitude / math.pow(speed,3)

In [10]:
# evaluation of the objective function along the parameter space. Coefficient values stil can be passed as paramters.

In [11]:
def parametric_coeffs_objective(inputX, alpha, beta):

    curvt = np.array([curvature(t2, alpha, beta) for t2 in inputX])

    return curvt

Optimization refers to minimizing/maximizing a function, for example, in linear regression we optimize for the intercept and coefficients of the model. The loss functions, also called objective function or cost function, is what we use to map the performance of a model to a real number ( actual value to optimize ). For our problem, this real number is the mean squared error (MSE), which is a common loss function.


$$
J(\theta_1, \theta_2)=\frac{1}{n}\sum^{n}_{i=1}(obj\_f(\theta_1, \theta_2)-y_i)^2
$$

The concept of the gradient plays a crucial role in gradient descent. However, if the model is unknown, we can still estimate the gradient using many methods. One effective approach for this purpose is the finite difference method, which we will discuss in relation to our objective function.

In [12]:
# calculates the mean squared error (MSE)

def cost_MSE(input_value, alpha, beta, target_value):

    predicted_value = parametric_coeffs_objective(input_value, alpha, beta)

    difference = (target_value - predicted_value)**2

    return sum(difference) / len(difference)

For example we have an objective function $J(\theta_1, \theta_2)$ where $\theta_1$ and $\theta_2$ are the parameters to be found. The general idea for gradient descent when optmizing these two values is as follows:

$$\theta_1(k+1) = \theta_1(k) - \alpha \frac{dJ} {d\theta_1}$$

$$\theta_2(k+1) = \theta_2(k) - \alpha \frac{dJ} {d\theta_2}$$

We lack the gradient since the analytical expression is lost while computing the objective function. To apply the finite difference method, we must consider the definition of the derivative in the limit. Then, numerical differentiation simply approximates it above using a very small $h$ value.

$$
\frac{df(x)}{dx} \equiv \lim_{h\rightarrow 0} \frac{f(x+h)-f(x)}{h}
$$

$$
\frac{df(x)}{dx} \approx \frac{f(x+h)-f(x)}{h}
$$

In [13]:
# calculates gradients using finite difference approximation.

def gradient_value_using_approx(inputX, outputY, alpha, beta):

    f = cost_MSE
    h = 0.001
    
    a_grad_val = (f(inputX, alpha+h, beta, outputY)-f(inputX, alpha, beta, outputY))/h
    b_grad_val = (f(inputX, alpha, beta+h, outputY)-f(inputX, alpha, beta, outputY))/h

    return (a_grad_val, b_grad_val)

In [14]:
# wrapper to fit the form in Gradient Descent:

def gradient_wrapper(inputX, outputY, values):
    
    return gradient_value_using_approx(inputX, outputY, values[0], values[1])

In [15]:
# gradient descent with callback function for printing and storing values (for example)

def gradient_descent(

    gradient, x, y,
    start,learn_rate=0.1, 
    n_iter=50, tolerance=1e-06,

    callback = lambda x : print(x)
):

    vector = start

    for _ in range(n_iter):

        diff = -learn_rate * np.array(gradient(x, y, vector))

        if np.all(np.abs(diff) <= tolerance):

            break

        vector += diff

        callback(np.copy(vector))

    return vector

In [17]:
# define the objective function: 
x = np.linspace(0, 2 * np.pi, 100)
y = parametric_coeffs_objective(x, 0.75, 0.5)

values = [] 

# execute gradient descent
gradient_descent(
    gradient_wrapper, x, y, 
    start=[1.2, 1.0], learn_rate=0.0001, n_iter=100,
    callback = lambda x : values.append(x)
)

# print iteration values;
print(values)

[array([1.20524894, 0.99703292]), array([1.21083597, 0.99412648]), array([1.21681526, 0.99127808]), array([1.22325386, 0.98848491]), array([1.23023638, 0.98574386]), array([1.23787217, 0.9830512 ]), array([1.24630644, 0.98040228]), array([1.25573859, 0.97779076]), array([1.26645376, 0.97520744]), array([1.27888142, 0.97263778]), array([1.29371393, 0.97005636]), array([1.31217689, 0.9674131 ]), array([1.33675712, 0.96459072]), array([1.37372976, 0.96122862]), array([1.44669519, 0.95545692]), array([1.72915847, 0.90926963]), array([2.05076265, 0.75640826]), array([2.06034582, 0.7588565 ]), array([2.07217912, 0.7617668 ]), array([2.08749771, 0.76534084]), array([2.10969466, 0.76996908]), array([2.14909177, 0.7766755 ]), array([2.28866218, 0.78929058]), array([1.75395072, 1.03765772]), array([4.97332863, 0.93133088]), array([50.00037578, 54.85343196]), array([3800432.42493974, -894297.28368339]), array([-3.51935719e+15, -2.32877795e+22])]


For our purposes, the actual plotting of the learning environments is as important here as the final value that optimizes our objective function. By plotting both sides, we can visualize the form emerging on the left and the objective function being learned on the right.

In [18]:
tetha = np.linspace(0, 2 * np.pi, 100)

In [19]:
import matplotlib.pyplot as plt

from matplotlib.collections import LineCollection  

from matplotlib.animation import FuncAnimation

In [20]:
cmap = plt.cm.get_cmap('jet')  

/tmp/ipykernel_370186/3779237694.py:1: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = plt.cm.get_cmap('jet')


In [21]:
def setup_fig_axs (obj_fun = np.array([])):

    fig, (axl, axr) = plt.subplots(
        
        ncols=2,
        figsize=(15, 5),
    )

    # ScalarMappable 
    sm = plt.cm.ScalarMappable(cmap=cmap) 
    
    fig.colorbar(sm, ax=axl) 

    # ax left
    lc_lft = LineCollection([], 
                            cmap=cmap, 
                            linewidth = 2.5)
    
    axl.add_collection(lc_lft)

    # ax right;
    ln_rgh, = axr.plot(tetha, tetha)
    axr.set_ylim([0, 80]) # fixed value;

    if (obj_fun.size > 0):

        axr.plot(tetha, obj_fun) # plotting t, b separately 

    axl.set_xlim(-1.5, 1.5)
    axl.set_ylim(-1.5, 1.5)

    return fig, lc_lft, sm, ln_rgh

In [22]:
def update(coeffs, lc_lft, sm, ln_rgh):

    t0_ = np.linspace(0, 2 * np.pi, 100)
    r0_ = np.array([f0_parametric(t_, coeffs[0], coeffs[1]) for t_ in t0_])

    render   = r0_.reshape(-1,1,2)
    segments = np.concatenate([render[:-1],render[1:]], axis=1)
    curvatur = np.array([curvature(t2, coeffs[0], coeffs[1]) for t2 in t0_])

    sm.set_array(curvatur.clip(0,1)) 

    lc_lft.set_segments(segments)
    lc_lft.set_array(curvatur.clip(0,1))
    lc_lft.cmap = cmap
    
    ln_rgh.set_ydata(curvatur)

    return ln_rgh, 

In [23]:
from IPython import display as disp

In [24]:

fig, lc, sm, line1 = setup_fig_axs(obj_fun= y)

ani = FuncAnimation(fig, 
                    update, 
                    fargs=[lc, sm, line1], 
                    frames= values, 
                    interval = 200, 
                    blit=True)

video = ani.to_html5_video()
html = disp.HTML(video)

disp.display(html) 
plt.close() 